<a href="https://colab.research.google.com/github/suchizz/doc_analyzer_frontend/blob/main/chatbot_theme_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required packages
!pip install -q fastapi uvicorn nest_asyncio pyngrok
!pip install -q langchain sentence-transformers faiss-cpu PyMuPDF transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00


In [2]:
!pip install -q langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [3]:
pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 3.6 MB/s eta 0:00:00


In [4]:
# ✅ Imports
import os
import fitz  # PyMuPDF
import nest_asyncio
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import uvicorn

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

In [5]:
# ✅ Initialize summarization model
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
summarizer = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [6]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import nltk
import tempfile
import os
nltk.download("punkt")
from nltk.tokenize import sent_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
from groq import Groq
import os

client = Groq(api_key="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")


In [8]:
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse
import os, tempfile
from typing import List
from groq import Groq
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import fitz  # PyMuPDF
from nltk.tokenize import sent_tokenize

In [9]:
app = FastAPI()

In [16]:
@app.post("/analyze")
async def analyze(files: List[UploadFile] = File(...), question: str = Form(...)):
    all_chunks = []

    # Extract text from PDFs and split into sentences with metadata
    for file in files:
        filename = file.filename
        temp_path = os.path.join(tempfile.gettempdir(), filename)
        with open(temp_path, "wb") as f:
            f.write(await file.read())

        pdf = fitz.open(temp_path)
        for page_num, page in enumerate(pdf, start=1):
            paragraphs = page.get_text("blocks")
            for para_idx, (_, _, _, _, text, _, _) in enumerate(paragraphs):
                sentences = sent_tokenize(text.strip())
                for sent_idx, sentence in enumerate(sentences, start=1):
                    if sentence.strip():
                        meta = {
                            "doc_name": filename,
                            "page": page_num,
                            "paragraph": para_idx + 1,
                            "sentence": sent_idx
                        }
                        all_chunks.append(Document(page_content=sentence, metadata=meta))

    # Handle case where no text is found
    if not all_chunks:
        return JSONResponse({
            "question": question,
            "direct_answers": [],
            "documents": [],
            "theme_summary": "⚠️ No text found in uploaded files."
        })

    # Generate embeddings and perform similarity search
    embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(all_chunks, embedder)
    results = db.similarity_search(question, k=6)

    answer_snippets = []
    doc_table = []
    combined_snippets = ""

    for doc in results:
        meta = doc.metadata
        citation = f"{meta['doc_name']} – Page {meta['page']}, Para {meta['paragraph']}, Sent {meta['sentence']}"
        snippet = f"🔹 {doc.page_content}\n📌 {citation}"
        answer_snippets.append(snippet)

        doc_table.append({
            "document": meta["doc_name"],
            "page": meta["page"],
            "paragraph": meta["paragraph"],
            "sentence": meta["sentence"],
            "answer": doc.page_content
        })

        combined_snippets += f"{citation}: {doc.page_content.strip()}\n"

    # Generate theme summary using Groq + LLaMA
    theme_prompt = f"""You are an expert assistant. Summarize the following extracted document sentences into 2–3 clear themes.
Each theme should include relevant citations and reflect what the documents are mainly discussing.

Snippets:
{combined_snippets}"""

    try:
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": theme_prompt}]
        )
        theme_summary = response.choices[0].message.content
    except Exception as e:
        theme_summary = f"⚠️ Theme generation failed: {str(e)}"

    return JSONResponse({
        "question": question,
        "direct_answers": answer_snippets,
        "documents": doc_table,
        "theme_summary": theme_summary
    })


In [17]:
# %%
# ✅ Launch ngrok tunnel
# Set your ngrok authtoken here. Replace "YOUR_AUTHTOKEN" with your actual authtoken.
# You can find your authtoken at https://dashboard.ngrok.com/get-started/your-authtoken
try:
    ngrok.set_auth_token("2yRs7KUrWwFDr5A2o6o354NwcAw_29PHY1wszJHELAEKAj8xT")
    public_url = ngrok.connect(8000)
    print(f"🔗 Your backend is live at: {public_url}")
except Exception as e:
    print(f"An error occurred while starting ngrok: {e}")
    print("Please ensure you have set your ngrok authtoken correctly.")

🔗 Your backend is live at: NgrokTunnel: "https://309c-34-57-16-2.ngrok-free.app" -> "http://localhost:8000"


In [18]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# ✅ Start FastAPI server
# ✅ Start FastAPI server
# Apply nest_asyncio patch to allow running asyncio in a Jupyter notebook
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [576]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    s

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
